In [1]:
import matplotlib.pyplot as plt
import numpy as np
from random import randint
import numpy
import timeit
import time
from math import ceil, floor, sqrt
from sklearn.linear_model import LinearRegression
from scipy import stats
import seaborn as sns

Below is implementation of standard multiplication algorithm.

Justification of its time complexity:
Standart multiplication has two loops, one of which is nested. Other then that all operations are elementary, therefore the time complexity is O(n^2)

In [ ]:
def simple_multipl(num1: int, num2:int) -> 0:
    res = 0
    n2=num2%10
    i=0
    while n2 != 0:
        sub_res = 0
        n1=num1%10
        j=0
        carry=0

        while n1 != 0:
            # print(n2, n1)
            single_digits_multiply = n1*n2+carry
            sub_res += (single_digits_multiply%10)*(10**j)
            carry = single_digits_multiply//10
            j+=1
            n1 = num1//(10**j)%10
        
        # print(sub_res)
        res+=sub_res*(10**i)
        i+=1
        n2=num2//(10**i)%10
    return res


Below is implementation of standard multiplication algorithm.

Justification of its time complexity:
The algorithm is recursive, and is described by formula

![](https://mathworld.wolfram.com/images/equations/KaratsubaMultiplication/NumberedEquation1.svg)

Time complexity of karatsuba multiplication is described by recursive equality T(n) = 3T(2/n)+O(n) - we replace 2 multiplications with only 1. Therefore time complexity is O(n^log3).

In [ ]:
def karatsuba(num1,num2):
    if num1 < 10 or num2 < 10:
        return num1*num2

    n = max(len(str(num1)), len(str(num2)))
    m = n//2

    n1_left  = num1 // 10**m
    n1_right = num1 % (10**m)

    n1_left = num2 // 10**m
    n1_right = num2 % (10**m)

    left_res = karatsuba(n1_left,n1_left)
    right_res = karatsuba(n1_right,n1_right)
    res = karatsuba(n1_left + n1_right, n1_left + n1_right) - left_res - right_res

    return int(left_res*(10**(m*2)) + res*(10**m) + right_res)

Test of time complexities of both algorithms

In [ ]:
testing_numbers = []
testing_lenghts = [10, 100, 1000, 3000, 6000]
for i in testing_lenghts:
    testing_numbers.append( (randint(10**(i-1), 10**i-1), randint(10**(i-1), 10**i-1)) )

In [ ]:
from timeit import default_timer

standart_times=[]
for num1, num2 in testing_numbers:

    # one_loop_times_st = []
    # for _ in range(500):

    #     t1 = time.time()
    #     simple_multipl(num1, num2)
    #     t2 = time.time()
    #     one_loop_times_st.append(t2-t1)
    
    # standart_times.append(np.quantile (one_loop_times_st, 0.1))
    times = []
    for _ in range(50):
        t1 = default_timer()
        simple_multipl(num1, num2)
        times.append(default_timer()-t1)

    standart_times.append(np.quantile(times, 0.1))

In [ ]:
from timeit import default_timer


karatsuba_times=[]

for num1, num2 in testing_numbers:

    # one_loop_times_kar = []
    # for _ in range(500):

    #     t1 = time.time()
    #     karatsuba(num1, num2)
    #     t2 = time.time()
    #     one_loop_times_kar.append(t2-t1)

    # karatsuba_times.append(np.mean(one_loop_times_kar))

    time_all = timeit.timeit(lambda: karatsuba(num1, num2), number=4, timer=default_timer)

    karatsuba_times.append(time_all/4)

Now let's see times of xecution for differen lenghts of numbers

In [ ]:
# print(standart_times)
# print(karatsuba_times)

plt.scatter(testing_lenghts, standart_times, color='green', label="Standart multiplication times")
plt.scatter(testing_lenghts, karatsuba_times, color='red', label="Karatsuba multiplication times")
plt.xlabel("number length")
plt.ylabel("execution times, seconds")
leg=plt.legend()

Here we try to take reverse function from theoretical O(n^2) dependency for standart multiplication and try to see if it fits to linear regression. Ideally values of sqrt(time) should be close to the line

In [ ]:
# model = LinearRegression()
# dots = np.array(testing_lenghts, [sqrt(i) for i in standart_times])
# model.fit(dots.reshape(-1, 1), zip(testing_lenghts, testing_lenghts))
# line = model.predict(dots)
# plt.plot(dots, line, color="red")
slope, intercept, r, p, std_err = stats.linregress(testing_lenghts, [sqrt(i) for i in standart_times])

def myfunc(x):
  return slope * x + intercept

mymodel = list(map(myfunc, testing_lenghts))

plt.scatter(testing_lenghts, [sqrt(i) for i in standart_times])
plt.plot(testing_lenghts, mymodel)

Here we try to take reverse function from theoretical O(n^(log3, 2)) dependency and try to see if it fits to linear regression. Ideally values of pow(time, 1/1.58) should be close to the line. Then it means that practical time complexity matches teoretical one.

In [ ]:
slope, intercept, r, p, std_err = stats.linregress(testing_lenghts, [pow(i, 1/1.58) for i in karatsuba_times])

def myfunc(x):
  return slope * x + intercept

mymodel = list(map(myfunc, testing_lenghts))

plt.scatter(testing_lenghts, [pow(i, 1/1.58) for i in karatsuba_times])
plt.plot(testing_lenghts, mymodel)

In [ ]:
same_l_dif_num = [1000, 1111, 6873, 9999]
times = [[]]*4
for i, n in enumerate(same_l_dif_num):
    for _ in range(150):
        t1 = default_timer()
        karatsuba(n,n)
        times[i].append(default_timer()-t1)
sns.histplot(times[0], color="blue", alpha=0.3)
sns.histplot(times[1], color="red", alpha=0.3)
sns.histplot(times[2], color="orange", alpha=0.2)
sns.histplot(times[3], color="green", alpha=0.2)

In [ ]:
same_l_dif_num = [1000, 1111, 6873, 9999]
times = [[]]*4
for i, n in enumerate(same_l_dif_num):
    for _ in range(250):
        t1 = default_timer()
        simple_multipl(n,n)
        times[i].append(default_timer()-t1)
sns.histplot(times[0], color="blue", alpha=0.3)
sns.histplot(times[1], color="red", alpha=0.3)
sns.histplot(times[2], color="orange", alpha=0.2)
sns.histplot(times[3], color="green", alpha=0.2)
  
